# **Sat4Ec**: Satellite data for economic flash estimation

The `sat4ec` automotive indicator is an algorithm for monitoring the production levels of automotive production sites in Germany using Sentinel-1 IW GRD data. Since cars parked in parking lots of automotive sites interact with the Sentinel-1 SAR wave, a strong or weak signal can be used to infer their presence or absence, respectively. The algorithm aggregates a single mean SAR backscatter value for each AOI and calculates a time series to identify times of high or low parking lot occupancy.

<img src="docs/readme_images/aoi_example.png" width="300">

Individual or all of the following automotive production sites can be selected:  <br>
<img src="docs/readme_images/map_facilities.png" width="300">
- *Audi sites* in Ingolstadt and Neckarsulm
- *BMW sites* in Dingolfing and Regensburg 
- *Ford sites* in Cologne and Saarlouis
- *Mercedes sites* producing cars in Bremen, Sindelfingen, and Rastatt as well as truck production sites in Düsseldorf and Ludwigsfelde
- *Opel sites* in Eisenach and Rüsselsheim
- *Porsche plant* in Leipzig
- *VW sites* producing cars in Emden, Wolfsburg and Zwickau as well as the truck production site in Hanover

For each plant, production parking lots can be analyzed individually or collectively on the basis of daily or monthly data. 

**Example Output for BMW Regensburg**

<img src="docs/readme_images/s1_pixels.png" width="250">
<img src="docs/readme_images/s1_pixels_mean.png" width="250">
<img src="docs/readme_images/bmw.png" width="455">

The following [peer-reviewed publication](https://doi.org/10.1109/JSTARS.2025.3601351) describes the methods and validation process in more detail:

<div class="alert alert-block alert-info">
Kraft, F., Martinis, S., Krullikowski, C., Plank, S., Anghelea, A., Delago Blasco, J. M., Schönenberger, K., Köhlmann, M., & Brauchler, M. (2025). Satellite Data for Economic Insights: Towards Tracking Automotive Production in Germany with Sentinel-1 for Economic Nowcasting. <i>IEEE Journal of Selected Topics in Applied Earth Observations and Remote Sensing</i>, 1–10. https://doi.org/10.1109/JSTARS.2025.3601351
</div>

### Running the algorithm

In [ ]:
from sat4ec.execution.exe_production import Production
from pathlib import Path
import os
import getpass
from IPython.display import display, Image
from sat4ec.execution.exe_config import Config

#### Sentinel Hub User Credentials:
**Prerequisite:** The `sat4ec` indicator requires an active [SentinelHub](https://www.sentinel-hub.com/) subscription or [Copernicus Data Space Ecosystem (CDSE)](https://dataspace.copernicus.eu/) registration.

Instructions for obtaining Sentinel Hub user credentials:
- directly using a **commercial Sentinel Hub access**: https://docs.sentinel-hub.com/api/latest/api/overview/authentication/ 
- through the **CDSE**: https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Overview/Authentication.html 

In [ ]:
# Choose data access via commercial SentinelHub ("commercial") or via the Copernicus Data Space Ecosystem ("cdse")
sh_env = input("Enter Sentinel Hub environment (commercial/cdse): ").strip().lower()

# Capture credentials from the user
sh_client_id = getpass.getpass("Enter your Sentinel Hub Client ID: ")
sh_client_secret = getpass.getpass("Enter your Sentinel Hub Secret Client ID: ")

# Validate credentials
if not sh_client_id or not sh_client_secret:
    raise ValueError("No valid Sentinel Hub credentials available. Please provide valid credentials.")

# Save the credentials as environment variables
os.environ["SH_CLIENT_ID"] = sh_client_id
os.environ["SH_CLIENT_SECRET"] = sh_client_secret
os.environ["SH_ENV"] = sh_env

#### Production Site Selection

In [ ]:
# choose one or more automotive facilties to analyze by commenting out all others

aois = [
    "audi_ingolstadt",
    "audi_neckarsulm",
    "bmw_dingolfing",
    "bmw_regensburg",
    "bmw_leipzig",
    "ford_cologne",
    "ford_saarlouis",
    "mercedes_bremen",
    "mercedes_duesseldorf",
    "mercedes_ludwigsfelde",
    "mercedes_rastatt",
    "mercedes_sindelfingen",
    "opel_eisenach",
    "opel_ruesselsheim",
    "porsche_leipzig",
    "vw_emden",
    "vw_hannover",
    "vw_wolfsburg",
    "vw_zwickau",
]

#### Output Parameters

In [ ]:
for aoi in aois:
    print(f"Preparing data for automotive facility {aoi}.")

    conf = Config(
        aoi_dir=Path(r"docs\aois"),
        working_dir=Path(r"sat4ec"), # path to your local working directory
        out_dir=Path(r"output"),
        orbit="asc",  # choose between ascending or descending orbit or both
        aoi=aoi,
        ext="geojson",
        start=None,  # specify a starting date for the time series (YYYY-MM-DD) or comment this line or enter None if using default start date
        end=None,  # specify an end date for the time series (YYYY-MM-DD) or comment this line or enter None if using automatic end date
        monthly=False, # choose between monthly (True) and daily (False) data
        regression="spline", # choose between available interpolation methods for daily data: spline (default), rolling mean, polynomial
        linear=True, # whether to plot linear regression or not
        linear_fill=False, # whether to plot linear insensitive area or not
        aoi_split=True, # choose between data for individual parking lots (True) or entire facilities (False)
        overwrite_raw=False, # whether to overwrite existing data or not
        online=True, # whether to load statistics from Sentinel Hub, False if solely using offline data
    )
    
    prod = Production(config=conf)
    prod.workflow()

### Displaying the plots

In [ ]:
def find_png_files(aoi):
    plot_location = os.path.join("output", aoi, "plot", "png")
    png_files = []
    if os.path.isdir(plot_location):
        for file in os.listdir(plot_location):
            if file.endswith(".png"):
                png_files.append(os.path.join(plot_location, file))
    return png_files

for aoi in aois:
    print("Plots for automotive facility", aoi)
    png_files = find_png_files(aoi)
    for png_file in png_files:
        display(Image(filename=png_file, width=800))